# Assignment 7: Anomaly Detection and AWS

## Objective

This assignment has two parts.

In the first part, you will be instructed to apply ** unsupervised ** learning to address a practical problem. For simplicity, we will use intrusion detection as an example. But the principle can be used in many other fields, such as fraud detection and medical care. After completing this assignment, you should be able to answer the following questions:

### Anomaly Detection

1. What's the difference between model development and model serving?
2. How to transform categorical features into numerical features?
3. How to derive anomalies from clustering results?
4. How to tune parameters for unsupervised learning?



In the second part, I designed a task aiming to deepen your understanding of AWS. After completing this task, you should be able to answer the following questions:


### AWS

1. Why Cloud Computing?
2. How to launch an Amazon EC2 Instance?
3. How to run an iPython Notebook Server on AWS - EC2 instance?
4. What are the key concepts of S3?
5. How to transfer files between S3 and a local machine?


All the datasets used in this assignment can be downloaded at [A7.zip](A7.zip).

## Part 1: Anomaly Detection

### Overview

In practice, when you are facing an ML problem, the process of solving it basically consists of two phases: _model development_ and _model serving_. 

* In model development, your job is to figure out what's the best algorithms, features, and parameters should be chosen based on historical data. This is often an iterative and off-line process. 

* Once you develop a satisfactory model, you will need to use the model to serve new requests and make predictions. This is often an on-line process, so you have to think about how to make the predictions as fast as possible and how to efficiently update the model when new data arrive. 

In this assignment, you are going to focus on the model development stage. Model serving is a hot research topic. There are many interesting open problems. If you are interested in them, I highly recommend you to take a look at [Clipper](https://rise.cs.berkeley.edu/blog/publication/clipper-low-latency-online-prediction-serving-system/), a new system that is being developed in the RISELab at UC Berkeley for serving machine learning predictions. 

### Model Development

Suppose you want to develop a model that can detect anomalous connections to your company's server. The server log contains all the information of historical connections; your nice colleague has already helped you to transform the raw log into a collection of feature vectors, where each feature vector characterizes a connection in 40 dimensions, e.g., number of failed login attempts, length (number of seconds) of the connection. Here is one example feature vector:
```
[udp,SF,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,240,0.94,0.01,0.00,0.00,0.00,0.00,0.00,0.00]
```

Your task is to take these feature vectors as input and develop an unsupervised-learning model to detect anomalous connections. In the lecture, we have gone through this process. In the assignment, you are going to implement two functions: <font color="blue">cat2Num</font> and <font color="blue">addScore</font>, by doing Task A and Task B, respectively.

``` python
# anomaly_detection.py
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import operator


class AnomalyDetection():
    
    def readData(self, filename):
        self.rawDF = spark.read.parquet(filename).cache()

    def cat2Num(self, df, indices):
        """
            Write your code!
        """

    def addScore(self, df):
        """
            Write your code!
        """
         
    def detect(self, k, t):
        #Encoding categorical features using one-hot.
        df1 = self.cat2Num(self.rawDF, [0, 1]).cache()
        df1.show()
        
        #Clustering points using KMeans
        features = df1.select("features").rdd.map(lambda row: row[0]).cache()
        model = KMeans.train(features, k, maxIterations=40, runs=10, initializationMode="random", seed=20)
        
        #Adding the prediction column to df1
        modelBC = sc.broadcast(model)
        predictUDF = udf(lambda x: modelBC.value.predict(x), StringType())
        df2 = df1.withColumn("prediction", predictUDF(df1.features)).cache()
        df2.show()
        
        #Adding the score column to df2; The higher the score, the more likely it is an anomaly 
        df3 = self.addScore(df2).cache()
        df3.show()    
      
        return df3.where(df3.score > t)
    
 
if __name__ == "__main__":
    ad = AnomalyDetection()
    ad.readData('data/logs-features-sample')
    anomalies = ad.detect(8, 0.97)
    print anomalies.count()
    anomalies.show()
```

### A toy dataset for testing

To test your program, you can read a toy dataset:

```python
    def readToyData(self):
        data = [(0, ["http", "udt", 0.4]), \
                (1, ["http", "udf", 0.5]), \
                (2, ["http", "tcp", 0.5]), \
                (3, ["ftp", "icmp", 0.1]), \
                (4, ["http", "tcp", 0.4])]
        schema = ["id", "rawFeatures"]
        self.rawDF = sqlCt.createDataFrame(data, schema)
        
```

After calling ``anomalies = ad.detect(2, 0.9)`` in the main func, your program should output the followings:

**df1.show():**

```
+---+----------------+-----------------------------------+
|id |rawFeatures     |features                           |
+---+----------------+-----------------------------------+
|0  |[http, udt, 0.4]|[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.4]|
|1  |[http, udf, 0.5]|[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.5]|
|2  |[http, tcp, 0.5]|[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.5]|
|3  |[ftp, icmp, 0.1]|[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.1]|
|4  |[http, tcp, 0.4]|[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.4]|
+---+----------------+-----------------------------------+
```

**df2.show():**

```
+---+----------------+-----------------------------------+----------+
|id |rawFeatures     |features                           |prediction|
+---+----------------+-----------------------------------+----------+
|0  |[http, udt, 0.4]|[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.4]|0         |
|1  |[http, udf, 0.5]|[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.5]|0         |
|2  |[http, tcp, 0.5]|[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.5]|0         |
|3  |[ftp, icmp, 0.1]|[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.1]|1         |
|4  |[http, tcp, 0.4]|[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.4]|0         |
+---+----------------+-----------------------------------+----------+
```

**df3.show():**

```
+---+----------------+-----------------------------------+----------+-----+
|id |rawFeatures     |features                           |prediction|score|
+---+----------------+-----------------------------------+----------+-----+
|0  |[http, udt, 0.4]|[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.4]|0         |0.0  |
|1  |[http, udf, 0.5]|[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.5]|0         |0.0  |
|2  |[http, tcp, 0.5]|[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.5]|0         |0.0  |
|3  |[ftp, icmp, 0.1]|[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.1]|1         |1.0  |
|4  |[http, tcp, 0.4]|[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.4]|0         |0.0  |
+---+----------------+-----------------------------------+----------+-----+
```

**anomalies.show():**
```
+---+----------------+-----------------------------------+----------+-----+
|id |rawFeatures     |features                           |prediction|score|
+---+----------------+-----------------------------------+----------+-----+
|3  |[ftp, icmp, 0.1]|[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.1]|1         |1.0  |
+---+----------------+-----------------------------------+----------+-----+
```

Note that you do not need to do feature scaling in this assignment, but  you should make sure that you know how to do it  (See [StandardScaler](https://spark.apache.org/docs/latest/ml-features.html#standardscaler)  for reference). 



### Task A. Categorical Features --> Numerical Features

As you can see from the above feature vector, the first two dimensions in each feature vector are categorical features. For example, the first dimension can be one of the following cases: “tcp”, “udp”, or, “icmp”. You can represent these categorical features using one-hot representation. In other words, the first dimension can be replaced with [1,0,0] for “tcp”, [0,1,0] for “udp”, and [0,0,1] for “icmp”. 

In Task A, your job is to implement the <font color="blue">cat2Num</font> function.

```python
    def cat2Num(self, df, indices):
        """ 
            Input: $df represents a DataFrame with two columns: "id" and "rawFeatures"
                   $indices represents which dimensions in $rawFeatures are categorical features, 
                    e.g., indices = [0, 1] denotes that the first two dimensions are categorical features.
                    
            Output: Return a new DataFrame that adds the "features" column into the input $df
            
            Comments: The difference between "features" and "rawFeatures" is that 
            the latter transforms all categorical features in the former into numerical features 
            using one-hot key representation
        """
    
```

**Note**.
1. You have to implement the one-hot encoding by yourself rather than use an existing implmentation in a libary. 
2. Note that in this function, you are not allowed to convert DataFrame to an RDD.


### Task B. Adding Anomaly Score for Each Data Point

As you may remember, the intuition of our anomaly detection approach was that clusters with a small number of data points will correspond to attacks or anomalies. We use this intuition to generate a confidence score from the clustering model’s output. The confidence score reflects how much the clustering model believes a data point is an attack or not. Let us assume $x$ is a data point describing a network connection. We can use:

$$score(x) = \frac{N_{max}-N_{x}}{N_{max}-N_{min}}$$

to score $x$ as being an anomaly. Note that in this equation, $N_{max}$ and $N_{min}$ reflect the size of the largest and smallest clusters, respectively. $N_{x}$ represents the size of the cluster assigned to $x$. If you check the equation carefully, you will notice that $score(x) = 1$ when $x$ is assigned to the smallest cluster and $score(x)$ = 0 when $x$ is assigned to a large cluster.


In Task B, your job is to implement the <font color="blue">addScore</font> function. **Note** that in this function, you are not allowed to convert DataFrame to an RDD.

```python
def addScore(self, df):
    """ 
        Input: $df represents a DataFrame with four columns: "id", "rawFeatures", "features", and "prediction"
        Output: Return a new DataFrame that adds the "score" column into the input $df

        To compute the score of a data point x, we use:

             score(x) = (N_max - N_x)/(N_max - N_min), 

        where N_max and N_min represent the size of the largest and smallest clusters, respectively,
              and N_x represents the size of the cluster assigned to x 
    """
```

### Task C. Parameter Tuning for Unsupervised Learning


As you have seen from Assignments 1 and 2, parameter tuning plays an important role in improving the model quality of **supervised learning**. In this assignment, your task is to figure out how to tune parameters for **unsupervised learning**. There are two parameters that need to be tuned: 

* $k$: the number of clusters
* $t$: the score threshold 

They are set to k = 8 and t = 0.97 in the above program. In fact, if you changed them to different values, the result could be quite different. Thus, it is important for you to know how to tune $k$ and $t$ in practice.  

In Task C, imagine yourself as a program manager (rather than a developer). Please write an email to tell a developer (named Nick) how you want to tune the parameters. YOU DO NOT NEED TO WRITE THE CODE!



## Part 2: Amazon Web Services (AWS)

Cloud computing is well known for a _pay-as-you-go_ model. That is, pay only for what you use. This model significantly reduces the cost of data storage and data processing, making big data techniques accessible to more and more people. Therefore, the company that you are going to work in the future is very likely to use EC2 instances for data processing and use S3 for data storage.   

To help you overcome this barrier, you will learn how to use AWS EC2 and S3 in Part 2. 



### Task A.1: Launching an Amazon EC2 Instance (Infrastructure)

Amazon provides an AWS Free Tier that includes services with a free tier available for 12 months. Please follow ["Getting Started with Amazon EC2 Linux Instances"](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html) to launch a free EC2 instance. 

<u>__Please choose the ubuntu OS system:__</u>
> <img src="img/ubuntu.png"/>

<u>__Please choose the free t2.micro:__</u>
> <img src="img/free.png"/>



### Task A.2 : Running an iPython Notebook Server on EC2 (Platform)

In Task A.2, your job is to install an iPython notebook on the above EC2 instance and access the notebook from your own laptop. You will find that this is a SUPER cool idea because this setup allows you to (1) conduct data analysis from anywhere and (2) have scalable storage/computational resources. In fact, this kind of setup is pretty common in industry right now.

There are many tutorials online. Please do a search by yourself, and follow the instructions to setup the environment. 

__Comments:__
* Pay attention to security group settings
* Install your python libraries through Anaconda Python Distribution
* Set the password to <font color="red">733</font>
* The URL should start with <font color="red">https</font>





### Task A.3: Transferring Files between S3 and Local Machine (Application)  

Amazon S3 (Simple Storage Service) is an online file storage web service. It allows for storing data in any format, and sharing data with anyone, and downloading data from anywhere. Most importantly, it's so cheap (a few cents per GB per month). 


You can use S3 either with a web-based management console, or full REST APIs and SDKs (e.g., [Boto 3](https://boto3.readthedocs.org/en/latest/)). They are all very easy to use. But please keep in mind some basic [concepts](http://docs.aws.amazon.com/AmazonS3/latest/dev/Introduction.html#CoreConcepts) when using them.



__Task Description:__

The goal of this task is to give you some sense on how the S3 console and the Boto S3 API work. Please follow the following steps to finish this task.

1. Download the NBA dataset to your local machine 
2. Create a bucket in your S3 using the S3 console (you can use any name for the bucket)
3. Upload "playbyplay20120510040.__zip__" to the bucket   
4. Create a new python notebook on your EC2 instance
5. In the notebook, please write a piece of code that can use the [Boto 3 API](https://boto3.readthedocs.org/en/latest/guide/migrations3.html) to do the followings:
  - Retrieve "playbyplay20120510040.zip" from __S3__ to your local machine
  - Extract the lines that contain the word of "bryant" (ignore case). You can use the [zipfile](https://docs.python.org/2/library/zipfile.html) library to read the zip file directly.
  - Write the extracted lines to a local file named "bryant.txt"
  - Upload "bryant.txt" to the bucket that you created 


__Hints:__

* Please first install boto3 into your EC2 instance by running the following command:
> conda install boto3

* You should never put *AWS_ACCESS_KEY_ID* and *AWS_SECRET_ACCESS_KEY* into your code. To avoid this, you can set them as system environment variables 

* Please take a look at the Boto 3 Transfer Manager [API](http://boto3.readthedocs.org/en/latest/guide/s3.html#using-the-transfer-manager)

__Submission:__

* Please submit a URL that links to the notebook your created


### Where To Go From Here (Optional)

There are a few things that you can do to improve the iPython Notebook setup:

1. Apply for [AWS Education Credits](https://aws.amazon.com/education/awseducate/) that allow you to launch a larger instance or a cluster of multiple instances for free
2. You can launch EC2 spot instances (as worker nodes) to save cost
3. Enable your iPython Notebook to support pyspark

## Submission

Implement <font color="blue">cat2Num</font> and <font color="blue">addScore</font> functions in `anomaly_detection.py`. Submit your code file (`anomaly_detection.py`), your email content (`email.txt`), and the notebook URL to the CourSys activity Assignment 7.